# 8: Erzeugen eigener POS-Tagger

- Mapping von POS-Tagsets
- Training eigener Tagger (mit Elternklasse nltk.tag.sequential.SequentialBackoffTagger)
- Verknüpfung und Speicherung (Serialisierung) von Taggern
- TreeTagger (Installation mit UNIX-Hilfsmitteln)


In [1]:
import sys
import nltk

In [2]:
# Ergebnis der Anwendung von xmltcf.py (altmann_elementarorganismen_1890) von vorheriger Sitzung
# zeilenweise in Liste einlesen:
with open('sents.txt') as f: 
    sents = f.readlines()
print(sents[42])

Max/NE Schultze/NE ,/$, Ueber/APPR Muskelkörperchen/NN und/KON das/PDS ,/$, was/PRELS man/PIS eine/ART Zelle/NN zu/PTKZU nennen/VVINF habe/VAFIN ./$.



In [6]:
# str2tuple: Hilfsfunktion zur Konvertierung "Token/POS"-Format in Tupel-Format:
print(nltk.tag.str2tuple("Fliege/NN"))

('Fliege', 'NN')


In [4]:
# Deserialisierung von sents.txt (Serialisierung in semistrukturiertem Austauschformat):

# Erzeugung von Tupellisten pro Satz/Zeile (ohne Zeilenumbruch: -1):
tagged_sents = [[nltk.tag.str2tuple(t) for t in sent[:-1].split(" ")] for sent in sents]
print(tagged_sents[42])

[('Max', 'NE'), ('Schultze', 'NE'), (',', '$,'), ('Ueber', 'APPR'), ('Muskelkörperchen', 'NN'), ('und', 'KON'), ('das', 'PDS'), (',', '$,'), ('was', 'PRELS'), ('man', 'PIS'), ('eine', 'ART'), ('Zelle', 'NN'), ('zu', 'PTKZU'), ('nennen', 'VVINF'), ('habe', 'VAFIN'), ('.', '$.')]


## Tagsets
* Tagsets sind oft inkompatibel zueinander
* [DTA](https://www.deutschestextarchiv.de) verwendet das [Stuttgart-Tübingen Tagset](https://www.sfs.uni-tuebingen.de/Elwis/stts/stts.html) (STTS)
* NLTK-Korpora verwenden verschiedene Tagset (z.B. [Penn Treebank Project](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html))
* viele Korpora erm&ouml;glichen die Konvertierung in das [Universal Tagset](https://arxiv.org/abs/1104.2086)
* Verwendung von Abbildungen (Mappings) zwischen Tagsets (z.B. [universal-pos-tags](https://github.com/slavpetrov/universal-pos-tags))

In [11]:
#nltk.help.upenn_tagset()
nltk.help.upenn_tagset("V")

VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...
VBD: verb, past tense
    dipped pleaded swiped regummed soaked tidied convened halted registered
    cushioned exacted snubbed strode aimed adopted belied figgered
    speculated wore appreciated contemplated ...
VBG: verb, present participle or gerund
    telegraphing stirring focusing angering judging stalling lactating
    hankerin' alleging veering capping approaching traveling besieging
    encrypting interrupting erasing wincing ...
VBN: verb, past participle
    multihulled dilapidated aerosolized chaired languished panelized used
    experimented flourished imitated reunifed factored condensed sheared
    unsettled primed dubbed desired ...
VBP: verb, present tense, not 3rd person singular
    predominate wrap resort sue twist spill cure lengthen brush terminate
 

In [74]:
print(nltk.corpus.brown.tagged_words()[:8])
print(nltk.corpus.brown.tagged_words(tagset='universal')[:8])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT')]
[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET')]


## Mapping von STTS nach Universal

In [9]:
# Max/NE Schultze/NE ,/$, Ueber/APPR Muskelkörperchen/NN und/KON das/PDS ,/$, was/PRELS man/PIS eine/ART Zelle/NN zu/PTKZU nennen/VVINF habe/VAFIN ./$.
def stts2universal(tag):
    mappings = {
        'APPR': 'ADP', # adpositions (preposition + postpositions)
        'KON': 'CONJ',
        'PDS': 'PRON',
        'PRELS': 'PRON',
        'PIS': 'PRON',
        'ART': 'DET',
        'PTKZU': 'PRT',
        # ...
    }
    if tag in mappings: 
        return mappings[tag]
    if tag.startswith('V'):
        return 'VERB'
    if tag.startswith('N'):
        return 'NOUN'
    if tag.startswith('$'):
        return '.'
    return 'X'

univ = [(w, stts2universal(t)) for (w, t) in tagged_sents[42]]
print(univ)

[('Max', 'NOUN'), ('Schultze', 'NOUN'), (',', '.'), ('Ueber', 'ADP'), ('Muskelkörperchen', 'NOUN'), ('und', 'CONJ'), ('das', 'PRON'), (',', '.'), ('was', 'PRON'), ('man', 'PRON'), ('eine', 'DET'), ('Zelle', 'NOUN'), ('zu', 'PRT'), ('nennen', 'VERB'), ('habe', 'VERB'), ('.', '.')]


## Kontext

In [12]:
def zu_v_v(sent):
    for (w1,_), (w2,t2), (w3,t3) in list(nltk.trigrams(sent)):
        if w1 == 'zu' and t2.startswith('V') and t3.startswith('V'):
            print(w1, w2, w3)

for sent in tagged_sents[:250]:
    zu_v_v(sent)

zu nennen habe
zu thun haben
zu ändern vermocht
zu erwarten ist
zu entdecken waren
zu sehen ist
zu folgen pflegt


## Mehrdeutige W&ouml;rter

- Wörter mit mehr als 3 POS-Tags

In [20]:
words2tags = nltk.ConditionalFreqDist([(w.lower(), t) for sent in tagged_sents for (w, t) in sent])
words2tags['zu']

FreqDist({'APPR': 120, 'PTKA': 11, 'PTKVZ': 4, 'PTKZU': 403})

In [19]:
for w in words2tags.conditions():
    if len(words2tags[w]) > 3:
        print(w, ' '.join(words2tags[w].keys()))

zu APPR PTKZU PTKA PTKVZ
s. NN FM.XY VVIMP FM
bis APPR ADV KON KOUS
a. APPRART NE XY PIS
de FM NE FM.LA FM.DA
anderer NN ADJA PIS NE
statt KOUI APPR NN PTKVZ
anat. NN XY ADJA NE
krehl VVIMP PTKVZ NN ADJD VVFIN


---
## H&auml;ufigste Tags

- als Baseline / Default-Tagger (letzter Backoff-Tagger)

In [21]:
tags2counts = nltk.FreqDist([t for sent in tagged_sents for (w, t) in sent])
print(tags2counts.most_common(3))

[('NN', 9667), ('ART', 5459), ('APPR', 3794)]


## DefaultTagger
https://www.nltk.org/api/nltk.tag.sequential.html#nltk.tag.sequential.DefaultTagger



In [22]:
default_tagger = nltk.DefaultTagger('NN')
default_tagger.evaluate(tagged_sents)

0.19881537543960678

---
## H&auml;ufigste Suffixe

In [24]:
suffixes2tags = nltk.ConditionalFreqDist([(w.lower()[-2:], t) for sent in tagged_sents for (w, t) in sent if len(w) > 2])
for suf in suffixes2tags.conditions()[:10]:
    print(suf, suffixes2tags[suf].most_common(2))

ie [('ART', 1455), ('PWAV', 245)]
en [('NN', 2581), ('ADJA', 1756)]
nd [('KON', 1001), ('VAFIN', 253)]
re [('ADJA', 194), ('NN', 180)]
on [('APPR', 504), ('NN', 239)]
rd [('VAFIN', 119), ('NE', 3)]
nn [('KOUS', 166), ('ADV', 104)]
it [('APPR', 368), ('NN', 246)]
ei [('APPR', 244), ('VAFIN', 30)]
xt [('NE', 2)]


## SequentialBackoffTagger
https://www.nltk.org/api/nltk.tag.sequential.html#nltk.tag.sequential.SequentialBackoffTagger

- abstrakte Tagger-Klasse
- DefaultTagger (s.o.) basiert auf (erbt von) SequentialBackoffTagger
- eigener Tagger über Implementierung der `choose_tag`-Methode (z.B. return None wenn kein Tag spezifizierbar)
- weitere Funktionalität (Handling von Backoff-Taggern etc.) übernimmt weiter der SequentialBackoffTagger 

In [25]:
print(nltk.tag.SequentialBackoffTagger.__doc__)


    An abstract base class for taggers that tags words sequentially,
    left to right.  Tagging of individual words is performed by the
    ``choose_tag()`` method, which should be defined by subclasses.  If
    a tagger is unable to determine a tag for the specified token,
    then its backoff tagger is consulted.

    :ivar _taggers: A list of all the taggers that should be tried to
        tag a token (i.e., self and its backoff taggers).
    


In [26]:
print(nltk.tag.SequentialBackoffTagger.choose_tag.__doc__)


        Decide which tag should be used for the specified token, and
        return that tag.  If this tagger is unable to determine a tag
        for the specified token, return None -- do not consult
        the backoff tagger.  This method should be overridden by
        subclasses of SequentialBackoffTagger.

        :rtype: str
        :type tokens: list
        :param tokens: The list of words that are being tagged.
        :type index: int
        :param index: The index of the word whose tag should be
            returned.
        :type history: list(str)
        :param history: A list of the tags for all words before *index*.
        


## Suffix Tagger

- eigener Tagger, basierend auf SequentialBackoffTagger
- erbt alle Methoden von Elternklasse (z.B. `evaluate()`)

In [33]:
class SuffixTagger(nltk.tag.sequential.SequentialBackoffTagger): #erben von abstrakter Klasse
    def __init__(self, suffixes, n, backoff=None): #Konstruktor
        super().__init__(backoff) #Super-Konstruktor des SequentialBackoffTagger: Handling Backoff Tagger
        self.suffixes = suffixes 
        self.n = n

    def choose_tag(self, tokens, index, history):
        if len(tokens[index]) > self.n:
            if tokens[index][-self.n:] in self.suffixes:
                return self.suffixes[tokens[index][-self.n:]]
        return None # let the backoff tagger handle all other cases

n = 2 #Suffixe der Länge 2
#Extraktion der Suffixe:
suffixes2tags = nltk.ConditionalFreqDist([(w[-n:], t) for sent in tagged_sents for (w, t) in sent if len(w) > n])
#Abbildung Suffixe auf häufigste Tags:
suffixes = {suf: tags.most_common(1)[0][0] for (suf, tags) in suffixes2tags.items() if len(tags) > 0}
#Erzeugung Suffix-Tagger:
suffix_tagger = SuffixTagger(suffixes, n, default_tagger)

#Evaluierung (Methode der Elternklasse):
suffix_tagger.evaluate(tagged_sents)

0.41618164243259365

---
## Unigramm und N-Gramm Tagger
* Unigramm Tagger betrachten immer nur einen Token
* N-Gramm Tagger betrachten einen Token und `N-1` Kontexttoken
* siehe [NLTK Buch 5. Kapitel](https://www.nltk.org/book_1ed/ch05.html) und [Symbolische Programmiersprache 20/21](https://cis-sp2021.github.io/crawling_pos.pdf)

<img src="https://www.nltk.org/images/tag-context.png" alt="Tagger Context">

### Aufteilung in Training- und Evaluierungsmengen

In [46]:
import random

size = int(len(tagged_sents) * 0.9)
random.shuffle(tagged_sents)
test_sents = tagged_sents[size:]
train_sents = tagged_sents[:size]
print("train set:", len(train_sents))
print("test set: ", len(test_sents))

train set: 1358
test set:  151


### Kombination von Taggern

In [47]:
print("default tagger:", default_tagger.evaluate(test_sents))
print("suffix tagger: ", suffix_tagger.evaluate(test_sents))

default tagger: 0.20026263952724885
suffix tagger:  0.40993652878091486


In [51]:
t_unigram = nltk.UnigramTagger(train_sents)
t_unigram.evaluate(test_sents)

0.8734952943751368

In [53]:
t_bigram = nltk.BigramTagger(train_sents)
t_bigram.evaluate(test_sents)

0.1346027577150361

In [48]:
t1 = nltk.UnigramTagger(train_sents, backoff=suffix_tagger)
t1.evaluate(test_sents)

0.9067629678266579

In [49]:
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t2.evaluate(test_sents)

0.9157364850076604

In [60]:
t2.tag(["zu", "thun", "haben"])

[('zu', 'PTKZU'), ('thun', 'VVINF'), ('haben', 'VAFIN')]

### Speicherung von Taggern
* Speicherung/Serialisierung von Objekten mit [pickle](https://docs.python.org/3/library/pickle.html)
- binäres Format für Python-Objekte
* pickle ist **nicht** sicher; niemals Objekte von unbekannten/unzuverl&auml;ssigen Quellen laden

In [61]:
#Speichern und Laden unseres t2-Taggers:
import pickle
with open('tagger.pickle', 'wb') as f:
    pickle.dump(t2, f)
with open('tagger.pickle', 'rb') as f:
    loaded_tagger = pickle.load(f)
loaded_tagger.evaluate(test_sents)

0.9157364850076604

## TreeTagger
* [TreeTagger](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/) probabilistierscher POS- und Lemma-Tagger von Helmut Schmid (CIS, LMU)
* beinhaltet verschiedene Tagger und [Chunker](https://en.wikipedia.org/wiki/Shallow_parsing) f&uuml;r verschiedene [Sprachen](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/#parfiles)
* deutscher Tagger/Chunker verwendet das [Stuttgart-Tübingen Tagset](https://www.sfs.uni-tuebingen.de/Elwis/stts/stts.html) (STTS)
* Installation muss manuell vorgenommen werden:
  * manuelle Installation des TreeTaggers
  * [Python Wrapper](https://treetaggerwrapper.readthedocs.io/en/latest/) mit pip installieren

### wget

| Name | wget - nicht-interaktives Werkzeug zum Herunterladen von Daten aus den Netz |
|:---|:---|
|Überblick| wget \[OPTION\]... \[URL\]... |
|Beschreibung | L&auml;dt Dateien und Verzeichnissb&auml;ume aus dem Netz herunter. |
| Wichtige Optionen: | |
| -o, --output-file `log-file` | schreibt Log-Nachrichten nach `log-file` und nicht nach `stderr` |
| -q, --quiet | unterdr&uuml;ckt Log-Nachrichten |
| -v, --verbose | ausf&uuml;hrliche Log-Nachrichten |
| -c, --continue | setzt das Herunterladen von partiellen Dateien fort |
| -O, --output-document `file` | schreibt die heruntergeladenen Dateien nach `file` |


### _alternativ:_

### curl

| Name | curl - Werkzeug zur Daten&uuml;bertragung |
|:---|:---|
|Überblick| curl \[OPTION\]... \[URL\]... |
|Beschreibung | Tool zum Transferieren von Daten von oder auf einen Server. |
| Wichtige Optionen: | |
| -o, --output `log-file` | schreibt Log-Nachrichten nach `log-file` und nicht nach `stderr` |
| -s, --silent | unterdr&uuml;ckt Log-Nachrichten |
| -O, --remote-name `file` | schreibt die heruntergeladenen Dateien nach `file` (unter dem selbem Namen wie wie die Remote Datei) |


In [5]:
%%bash
wget -qO /dev/stdout https://cis-kb21.github.io | sed 15q

<!DOCTYPE html>
<html lang="en-US">

  <head>
    <meta charset='utf-8'>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width,maximum-scale=2">
    <link rel="stylesheet" type="text/css" media="screen" href="/assets/css/style.css?v=be470d2329bb63c0c069a513bb9bed4b1ab5fd8b">

<!-- Begin Jekyll SEO tag v2.7.1 -->
<title>Kurs “Korpusbearbeitung” | cis-kb21.github.io</title>
<meta name="generator" content="Jekyll v3.9.0" />
<meta property="og:title" content="Kurs “Korpusbearbeitung”" />
<meta property="og:locale" content="en_US" />
<meta name="description" content="Website for Korpusbearbeitung SoSe 21" />


### tar

| Name | tar - Archivierungswerkzeug |
|:---|:---|
|Überblick| tar -x \[-f ARCHIVE\] \[OPTION\]... \[FILE\]... |
|         | tar -t \[-f ARCHIVE\] \[OPTION\]... \[FILE\]... |
|         | tar -c \[-f ARCHIVE\] \[OPTION\]... \[FILE\]... |
|Beschreibung | Fasst mehrere Dateien in einer Datei zusammen und |
|             | komprimiert sie (`-c`), listet die Dateien in einem |
|             | Archiv auf (`-t`) oder extrahiert die Dateien aus |
|             | einem Archiv (`-x`). |
| Wichtige Optionen: | |
| -z, --gzip, --gunzip --ungzip | komprimiert die Archive mit `gzip(1)` |
| -Z, --compress, --uncompress | komprimiert die Archive mit `compress(1)` |
| -j, --bzip2 | komprimiert die Archive mit `bzip2(1)` |
| -J, --xz | komprimiert die Archive mit `xz(1)` |
| -C, --directory `DIR` | Wechselt nach `DIR` bevor irgendwelche Arbeiten |
|                       | durchgef&uuml;hrt werden |

In [20]:
%%bash
tar -czf ar.tar.gz sents.txt 08_pos_tagging.ipynb
echo "files in ar.tar.gz:"
tar -tzf ar.tar.gz | awk '{print "-> " $0}'
echo "file sizes:"
ls -hl sents.txt 08_pos_tagging.ipynb ar.tar.gz | awk '{print "-> " $9 ": " $5}'

files in ar.tar.gz:
-> sents.txt
-> 08_pos_tagging.ipynb
file sizes:
-> 08_pos_tagging.ipynb: 37K
-> ar.tar.gz: 138K
-> sents.txt: 516K


### Weitere Werkzeuge
* [curl](https://curl.se): Werkzeug zur Daten&uuml;bertragung ([Everything curl](https://curl.se/book.html))
* zcat: cat auf gezippten Dateien
* gzip: komprimiert Dateien gzip
* gunzip: dekomprimiert gzip Dateien

### Installation des TreeTagger
* Herunterladen des TreeTaggers
* Herunterladen der Hilfs-Skripte
* Herunterladen des Installations-Skripts
* Herunterladen der [Sprachparameterdateien](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/#parfiles)

In [205]:
%%bash
mkdir -p tree-tagger && cd tree-tagger
wget -c -q https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz # tree tagger
# wget -c -q https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger-MacOSX-3.2.3.tar.gz
wget -c -q https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz # scripts
wget -c -q https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/install-tagger.sh # install script
wget -c -q https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/german.par.gz # language parameter file

tar -xzf tree-tagger-linux-3.2.4.tar.gz
bash install-tagger.sh


Tagging scripts installed.
German parameter file installed.
Path variables modified in tagging scripts.

You might want to add /home/flo/devel/work/kb21/jupyter/08/tree-tagger/cmd and /home/flo/devel/work/kb21/jupyter/08/tree-tagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.

mkdir: cannot create directory ‘cmd’: File exists
mkdir: cannot create directory ‘lib’: File exists
mkdir: cannot create directory ‘bin’: File exists
mkdir: cannot create directory ‘doc’: File exists


In [73]:
%%bash
#Alternativ: mit curl (hier für MacOS)
mkdir -p tree-tagger && cd tree-tagger
#curl -O --silent https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
curl -O --silent https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger-MacOSX-3.2.3.tar.gz
curl -O --silent https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz # scripts
curl -O --silent https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/install-tagger.sh # install script
curl -O --silent https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/german.par.gz # language parameter file

#tar -xzf tree-tagger-linux-3.2.4.tar.gz
tar -xzf tree-tagger-MacOSX-3.2.3.tar.gz
bash install-tagger.sh


TreeTagger version for Mac OS-X installed.
Tagging scripts installed.
German parameter file installed.
Path variables modified in tagging scripts.

You might want to add ~/08_pos_tagging/tree-tagger/cmd and ~/08_pos_tagging/tree-tagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.



mkdir: cmd: File exists
mkdir: bin: File exists
mkdir: doc: File exists


In [77]:
%%bash
echo "Dies ist nur ein Test." | tree-tagger/cmd/tree-tagger-german

Dies	PDS	dies
ist	VAFIN	sein
nur	ADV	nur
ein	ART	eine
Test	NN	Test
.	$.	.


	reading parameters ...
	tagging ...
	 finished.


In [75]:
#!{sys.executable} -m pip install --upgrade pip treetaggerwrapper
import treetaggerwrapper as TTW  

In [76]:
tagger = TTW.TreeTagger(TAGLANG='de', TAGDIR='./tree-tagger')
tags = tagger.tag_text("Zwei Dinge sind unendlich, das Universum und die menschliche Dummheit, aber bei dem Universum bin ich mir noch nicht ganz sicher.")
print(tags)

['Zwei\tCARD\tzwei', 'Dinge\tNN\tDing', 'sind\tVAFIN\tsein', 'unendlich\tADJD\tunendlich', ',\t$,\t,', 'das\tART\tdie', 'Universum\tNN\tUniversum', 'und\tKON\tund', 'die\tART\tdie', 'menschliche\tADJA\tmenschlich', 'Dummheit\tNN\tDummheit', ',\t$,\t,', 'aber\tADV\taber', 'bei\tAPPR\tbei', 'dem\tART\tdie', 'Universum\tNN\tUniversum', 'bin\tVAFIN\tsein', 'ich\tPPER\tich', 'mir\tPRF\tich', 'noch\tADV\tnoch', 'nicht\tPTKNEG\tnicht', 'ganz\tADV\tganz', 'sicher\tADJD\tsicher', '.\t$.\t.']


## Übungsaufgaben 8

### Aufgabe 1 (Vergleich von POS-Taggern)

Laden Sie vier weitere
[TCF-kodierte](https://weblicht.sfs.uni-tuebingen.de/weblichtwiki/index.php/The_TCF_Format)
Dateien herunter
(z.B. [Drude](https://www.deutschestextarchiv.de/book/download_fulltcf/16377),
[Sachs](https://www.deutschestextarchiv.de/book/download_fulltcf/16178),
[Brehm](https://www.deutschestextarchiv.de/book/download_fulltcf/25157)
und
[Bölsche](https://www.deutschestextarchiv.de/book/download_fulltcf/16552)).
Verwenden Sie diese Dateien und die von letzter Woche
([Altmann](https://www.deutschestextarchiv.de/book/download_fulltcf/16299))
um wie heute in der Vorlesung einen Bigramm-Tagger auf einer
Trainingsmenge zu trainieren und auf einer Testmenge auszuwerten.

Installieren Sie den TreeTagger und den Treetaggerwrapper und
schreiben Sie eine entsprechende Tagger-Klasse, die Token mit Hilfe
des TreeTaggers annotiert.  Werten Sie diesen Tagger auf der
Trainingsmenge aus und vergleichen Sie die Ergebnisse mit denen des
Bigramm-Taggers.
